In [6]:
import dpkt
import socket

In [10]:
import dpkt

def read_pcap(filename):
    """Reads a pcap file using dpkt and extracts packet information."""
    with open(filename, 'rb') as f:
        pcap_reader = dpkt.pcap.Reader(f)
        packets = []
        for ts, pkt in pcap_reader:
            # Extract information from each packet (timestamp, protocol, etc.)
            # You can access packet data using dpkt's classes (Ethernet, IP, TCP, UDP)
            packets.append(get_packet_info(ts, pkt))
    return packets
            
def get_packet_info(ts, buf):
  """
  Extracts information from a single packet.
  """
  eth = dpkt.ethernet.Ethernet(buf)
  info = {
      'timestamp': ts,  # Capture timestamp
      'mac_src': eth.src.hex(),  # Source MAC address
      'mac_dst': eth.dst.hex(),  # Destination MAC address
      'ether_type': eth.type,  # Ethernet packet type (e.g., IP)
  }

  # Handle different packet types (expand as needed)
  if eth.type == dpkt.ethernet.ETH_TYPE_IP:
    ip = eth.data
    info.update({
      'ip_src': socket.inet_ntoa(ip.src),  # Source IP address
      'ip_dst': socket.inet_ntoa(ip.dst),  # Destination IP address
      'ip_proto': ip.p,  # IP protocol (e.g., TCP, UDP)
    })
    info.update({
        'data' : ip
    })
    # Further processing for TCP, UDP, etc. (see below)

  # Return dictionary containing extracted information
  return info #ip means data 

# Example usage
read_pcap('/home/perm/example/advance_python/pcap/ETH_IPv4_TCP_syn.pcap')


[{'timestamp': 1413578829.242339,
  'mac_src': '74e50bccfc50',
  'mac_dst': '001dd46257e1',
  'ether_type': 2048,
  'ip_src': '10.0.0.11',
  'ip_dst': '157.166.226.25',
  'ip_proto': 6,
  'data': IP(len=52, id=18701, df=1, ttl=128, p=6, sum=10220, src=b'\n\x00\x00\x0b', dst=b'\x9d\xa6\xe2\x19', opts=b'', data=TCP(sport=51384, dport=80, seq=234124756, off=8, win=8192, sum=30842, opts=b'\x02\x04\x05\xb4\x01\x03\x03\x02\x01\x01\x04\x02'))}]

In [ ]:
from scapy.all import rdpcap
import pandas as pd

# Read the PCAP file
packets = rdpcap('/home/perm/example/advance_python/pcap/ETH_IPv4_TCP_syn.pcap')

# Function to extract important header information
def extract_packet_info(packet):
    packet_info = {
        'src_ip': packet[0][1].src if packet.haslayer('IP') else None,
        'dst_ip': packet[0][1].dst if packet.haslayer('IP') else None,
        'src_port': packet[0][2].sport if packet.haslayer('TCP') or packet.haslayer('UDP') else None,
        'dst_port': packet[0][2].dport if packet.haslayer('TCP') or packet.haslayer('UDP') else None,
        'protocol': packet[0][1].proto if packet.haslayer('IP') else None,
        'length': len(packet)
    }
    return packet_info

# Extract information from each packet
packet_data = [extract_packet_info(packet) for packet in packets]

# Convert to DataFrame
df = pd.DataFrame(packet_data)

# Drop rows with None values
df.dropna(inplace=True)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file (optional)
df.to_csv('packet_data.csv', index=False)
